In [6]:
import sys
sys.path.append('/Users/pedro/Documents/GitHub/BBBdatagetsbetter')

from dotenv import load_dotenv
import os
import pandas as pd
from BBB_Library.data_fetcher import GooglePlacesFetcher
from BBB_Library.data_comparator import DataComparator
from BBB_Library.data_formatter import format_address, format_phone_number, format_zip


In [17]:
def main():
    load_dotenv()
    api_key = os.getenv('GOOGLE_API_KEY')
    fetcher = GooglePlacesFetcher(api_key)

    # Load business data from CSV
    df = pd.read_csv("/Users/pedro/Documents/GitHub/BBBdatagetsbetter/Data_Processing/Output_Data/cleaned_and_normalized_data_all.csv", nrows=25)

    df['phone'] = df['phone'].apply(format_phone_number)
    df['zip'] = df['zip'].apply(format_zip)

    # Create a list to store updated rows
    updated_rows = []

    # Iterate through each business entry and verify
    for _, row in df.iterrows():

        company_name = row["company_name"]
        city = row["city"]
        state = row["state"]
        address = row["address"]
        zip_code = row["zip"]
        phone = row["phone"]
        formatted_address = format_address(address, city, state, zip_code)
        print(formatted_address)

        # Construct query for Google Places API
        query = f"{company_name} {city}, {state}"

        # Retrieve business information from API
        api_data = fetcher.fetch_data(query)
        print(api_data)


        updated_row = row.copy()
        if api_data:
            confidence_score = DataComparator.calculate_confidence(row, formatted_address, api_data)

            # If business is not found in Google Places
            if confidence_score == -1:
                updated_row["outdated"] = True
                updated_row["outdated_reason"] = "No match found in Google Places"
            else:
                updated_row['confidence_score'] = confidence_score
                updated_row = DataComparator.update_outdated_info(updated_row, formatted_address, api_data, confidence_score)
        else:
            updated_row['outdated'] = True
            updated_row['outdated_reason'] = "No data found in the API"
        updated_rows.append(updated_row)

    updated_df = pd.DataFrame(updated_rows)
    #updated_df.to_csv("/Users/pedro/Documents/GitHub/BBBdatagetsbetter/BBB_Library/Data/updated_business_data.csv", index=False)
    #print("Business information verification complete. Updated data saved to 'updated_business_data.csv'")
    return updated_df

In [18]:
df = main()
df

78 Acker St E, Saint Paul, MN 55117
{'formatted_address': '78 Acker St E, St Paul, MN 55117, USA', 'formatted_phone_number': '(651) 222-4355', 'name': 'Able Fence Inc.', 'place_id': 'ChIJ-aYwa1bVslIR85ptHKgDQeY'}
Debugging calculate_confidence entry:
Local Name: Able Fence, Inc.
API Data Name: Able Fence Inc.
Comparing:
Able Fence, Inc.
Able Fence Inc.
78 Acker St E, Saint Paul, MN 55117
78 Acker St E, St Paul, MN 55117
6512224355
6512224355
2200 Nicollet Ave, Minneapolis, MN 55404
{'formatted_address': '2701 Central Ave NE, Minneapolis, MN 55418, USA', 'formatted_phone_number': '(612) 789-3535', 'name': 'Billman-Hunt Funeral Chapel', 'website': 'https://www.billmanhunt.com/prearrangements-form', 'place_id': 'ChIJefLZTOgts1IRfGVHUdkRoB0'}
Debugging calculate_confidence entry:
Local Name: Albin Chapel
API Data Name: Billman-Hunt Funeral Chapel
6855 Rowland Rd, Eden Prairie, MN 55344
{'formatted_address': '16394 Glory Ln, Eden Prairie, MN 55344, USA', 'formatted_phone_number': '(952) 949

,firm_id,company_name,address,city,state,zip,phone,email,url,confidence_score,outdated,outdated_reason,google_name,google_address,google_phone,google_url,google_place_id
0,2.0,"Able Fence, Inc.",78 Acker St E,Saint Paul,MN,55117,6512224355,afinc1008@qwestoffice.net,NaN,98.0,False,Information current,Able Fence Inc.,"78 Acker St E, St Paul, MN 55117",6512224355,,ChIJ-aYwa1bVslIR85ptHKgDQeY
1,5.0,Albin Chapel,2200 Nicollet Ave,Minneapolis,MN,55404,,office@albinchapel.com,http://www.albinchapel.com/,NaN,True,No match found in Google Places,NaN,NaN,NaN,NaN,NaN
2,5.0,Albin Funeral Chapel Inc,6855 Rowland Rd,Eden Prairie,MN,55344,9529149410,jimalbinson@gmail.com,NaN,NaN,True,No match found in Google Places,NaN,NaN,NaN,NaN,NaN
3,5.0,"Albin Endeavor, Inc.",PO Box 46147,Eden Prairie,MN,55344,9529140043,NaN,NaN,NaN,True,No match found in Google Places,NaN,NaN,NaN,NaN,NaN
4,7.0,Albrecht Company,1408 County Road C W,Roseville,MN,55113,6516334510,mail@albrechtcompany.com,https://www.albrechtcompany.com/,97.0,False,Information current,Albrecht Company,"1408 County Rd C West, Roseville, MN 55113",6516334510,http://www.albrechtcompany.com/,ChIJBV4IX8Mrs1IRSG2dCjvsIhw
5,7.0,"Albrecht Enterprises, LLC",NaN,NaN,NaN,,,edward@albrechtcompany.com,NaN,NaN,True,No match found in Google Places,NaN,NaN,NaN,NaN,NaN
6,9.0,Arthur Williams Opticians,366 Saint Peter St,Saint Paul,MN,55102,6516451976,arthurwilliamsoptical@gmail.com,http://www.arthurwilliamsoptical.com/,31.0,True,"Address mismatch, Phone mismatch",Arthur Williams Optical,"772 Cleveland Ave S, St Paul, MN 55116",6512242883,http://www.arthurwilliamsoptical.com/,ChIJNxZsCHgp9ocRPVr8wNPs6Gw
7,9.0,Arthur Williams Optical Inc,772 Cleveland Ave S,Saint Paul,MN,55116,7632242883,NaN,NaN,48.0,True,Phone mismatch,Arthur Williams Optical,"772 Cleveland Ave S, St Paul, MN 55116",6512242883,http://www.arthurwilliamsoptical.com/,ChIJNxZsCHgp9ocRPVr8wNPs6Gw
8,14.0,Albrechts,680 Cleveland Ave S,Saint Paul,MN,55116,,NaN,NaN,NaN,True,No match found in Google Places,NaN,NaN,NaN,NaN,NaN
9,18.0,Andersen Corporation,100 4th Ave N,Bayport,MN,55003,6513513022,donna.dingle@andersencorp.com,http://www.andersencorp.com,NaN,True,No match found in Google Places,NaN,NaN,NaN,NaN,NaN
